# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
# PREPROCESSING
from sklearn.preprocessing import LabelEncoder
# MODEL
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
# RANDOM OVERSAMPLER
from imblearn.over_sampling import RandomOverSampler
# SYNTHETIC OVERSAMPLING (SMOTE)
from imblearn.over_sampling import SMOTE
# CLUTER CENTROIDS
from imblearn.under_sampling import ClusterCentroids
# SMOTEENN
from imblearn.combine import SMOTEENN


# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Encode labels:
# Search for string columns to be converted to numerical types
for i in df.columns:
    if df[i].dtypes == object:
        print(i)

home_ownership
verification_status
issue_d
loan_status
pymnt_plan
initial_list_status
next_pymnt_d
application_type
hardship_flag
debt_settlement_flag


In [6]:
# Check the content of the above calculated String columns
print(f'* home_ownership:\n{df["home_ownership"].describe()}\n{df["home_ownership"].unique()}\n')
print(f'* verification_status:\n{df["verification_status"].describe()}\n{df["verification_status"].unique()}\n')
print(f'* issue_d:\n{df["issue_d"].describe()}\n{df["issue_d"].unique()}\n')
print(f'* loan_status:\n{df["loan_status"].describe()}\n{df["loan_status"].unique()}\n')
print(f'* pymnt_plan:\n{df["pymnt_plan"].describe()}\n{df["pymnt_plan"].unique()}\n')
print(f'* initial_list_status:\n{df["initial_list_status"].describe()}\n{df["initial_list_status"].unique()}\n')
print(f'* next_pymnt_d:\n{df["next_pymnt_d"].describe()}\n{df["next_pymnt_d"].unique()}\n')
print(f'* application_type:\n{df["application_type"].describe()}\n{df["application_type"].unique()}\n')
print(f'* hardship_flag:\n{df["hardship_flag"].describe()}\n{df["hardship_flag"].unique()}\n')
print(f'* debt_settlement_flag:\n{df["debt_settlement_flag"].describe()}\n{df["debt_settlement_flag"].unique()}\n')

* home_ownership:
count        68817
unique           4
top       MORTGAGE
freq         36219
Name: home_ownership, dtype: object
['RENT' 'MORTGAGE' 'OWN' 'ANY']

* verification_status:
count            68817
unique               3
top       Not Verified
freq             32895
Name: verification_status, dtype: object
['Source Verified' 'Verified' 'Not Verified']

* issue_d:
count        68817
unique           3
top       Jan-2019
freq         31041
Name: issue_d, dtype: object
['Mar-2019' 'Feb-2019' 'Jan-2019']

* loan_status:
count        68817
unique           2
top       low_risk
freq         68470
Name: loan_status, dtype: object
['low_risk' 'high_risk']

* pymnt_plan:
count     68817
unique        1
top           n
freq      68817
Name: pymnt_plan, dtype: object
['n']

* initial_list_status:
count     68817
unique        2
top           w
freq      60292
Name: initial_list_status, dtype: object
['w' 'f']

* next_pymnt_d:
count        68817
unique           2
top       May-2019
fre

In [7]:
# Sklearn default encoding
le = LabelEncoder()
dfEncoded = df.copy()
dfEncoded['home_ownership'] = le.fit_transform(dfEncoded['home_ownership'])
dfEncoded['verification_status'] = le.fit_transform(dfEncoded['verification_status'])
dfEncoded['pymnt_plan'] = le.fit_transform(dfEncoded['pymnt_plan'])
dfEncoded['initial_list_status'] = le.fit_transform(dfEncoded['initial_list_status'])
dfEncoded['application_type'] = le.fit_transform(dfEncoded['application_type'])       
dfEncoded['hardship_flag'] = le.fit_transform(dfEncoded['hardship_flag'])       
dfEncoded['debt_settlement_flag'] = le.fit_transform(dfEncoded['debt_settlement_flag'])       
dfEncoded.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [8]:
# Encode Date columns : issue_d and next_pymnt_d
dates = {
    "Jan-2019" : 201901,
    "Feb-2019" : 201902,
    'Mar-2019' : 201903,
    'Apr-2019' : 201904,
    'May-2019' : 201905
}
# Apply a lambda function
dfEncoded['issue_d'] = dfEncoded['issue_d'].apply(lambda x: dates[x])
dfEncoded['next_pymnt_d'] = dfEncoded['next_pymnt_d'].apply(lambda x: dates[x])
dfEncoded.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,201903,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,201903,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,201903,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,201903,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,201903,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


# Split the Data into Training and Testing

In [9]:
# Create our features 
X = dfEncoded.drop("loan_status",axis=1)

# Create our target
y = dfEncoded["loan_status"]

In [10]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,201901.726172,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.743862,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,201901.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,201901.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,201902.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,201902.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,201903.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [11]:
# Check the balance of our target values
y.value_counts() # => Highly imbalanced


low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [12]:
# Create X_train, X_test, y_train, y_test
X_trainOrig, X_testOrig, y_train, y_test = train_test_split(X, y, random_state=1)

# Scale data for better models comparison

# Creating a StandardScaler instance.
scaler = StandardScaler()

# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_trainOrig)

# Scaling the data.
X_train = X_scaler.transform(X_trainOrig)
X_test = X_scaler.transform(X_testOrig)

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [40]:
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampledRand, y_resampledRand = ros.fit_resample(X_train, y_train)
Counter(y_resampledRand) 

Counter({'low_risk': 51366, 'high_risk': 51366})

In [41]:
# Train the Logistic Regression model using the resampled data

# Logistic Regression model
model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

model.fit(X_resampledRand, y_resampledRand)

y_pred = model.predict(X_test)

In [42]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred) 

0.8381112286860117

In [43]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual high_r", "Actual low_r"], columns=["Predicted high_r", "Predicted low_r"])
cm_df

,Predicted high_r,Predicted low_r
Actual high_r,84,17
Actual low_r,2659,14445


In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.83      0.84      0.06      0.84      0.70       101
   low_risk       1.00      0.84      0.83      0.92      0.84      0.70     17104

avg / total       0.99      0.84      0.83      0.91      0.84      0.70     17205



### SMOTE Oversampling

In [36]:
# Resample the training data with SMOTE
X_resampledSMOTE, y_resampledSMOTE = SMOTE(random_state=1,
sampling_strategy='auto').fit_resample(
   X_train, y_train)
Counter(y_resampledSMOTE)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [37]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

model.fit(X_resampledSMOTE, y_resampledSMOTE)

y_pred = model.predict(X_test)

In [38]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.832701979271828

In [39]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual high_r", "Actual low_r"], columns=["Predicted high_r", "Predicted low_r"])
cm_df

,Predicted high_r,Predicted low_r
Actual high_r,81,20
Actual low_r,2336,14768


In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.80      0.86      0.06      0.83      0.69       101
   low_risk       1.00      0.86      0.80      0.93      0.83      0.70     17104

avg / total       0.99      0.86      0.80      0.92      0.83      0.70     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [25]:
# Resample the data using the ClusterCentroids resampler
cc = ClusterCentroids(random_state=1)
X_resampledCC, y_resampledCC = cc.fit_resample(X_train, y_train)
Counter(y_resampledCC)

Counter({'high_risk': 246, 'low_risk': 246})

In [26]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

model.fit(X_resampledCC, y_resampledCC)

y_pred = model.predict(X_test)

In [27]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.8036074011984922

In [28]:
# Display the confusion matrix
matrix = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual high_r", "Actual low_r"], columns=["Predicted high_r", "Predicted low_r"])
cm_df

,Predicted high_r,Predicted low_r
Actual high_r,86,15
Actual low_r,4178,12926


In [30]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.85      0.76      0.04      0.80      0.65       101
   low_risk       1.00      0.76      0.85      0.86      0.80      0.64     17104

avg / total       0.99      0.76      0.85      0.86      0.80      0.64     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampledSMOTEENN, y_resampledSMOTEENN = smote_enn.fit_resample(X, y)
Counter(y_resampledSMOTEENN)

Counter({'high_risk': 68458, 'low_risk': 62022})

In [32]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=1, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
model.fit(X_resampledSMOTEENN, y_resampledSMOTEENN)

y_pred = model.predict(X_test)

In [33]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.6008287100927118

In [34]:
# Display the confusion matrix
matrix= confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    matrix, index=["Actual high_r", "Actual low_r"], columns=["Predicted high_r", "Predicted low_r"])
cm_df


,Predicted high_r,Predicted low_r
Actual high_r,87,14
Actual low_r,11284,5820


In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.86      0.34      0.02      0.54      0.31       101
   low_risk       1.00      0.34      0.86      0.51      0.54      0.28     17104

avg / total       0.99      0.34      0.86      0.50      0.54      0.28     17205

